In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import json

# Check if the connection to the LLM works

In [2]:
# load env vars
load_dotenv()
api_key = os.getenv("OPENROUTER_API_KEY")

In [3]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key
)

In [5]:
print("Testing connection to OpenRouter...")

try:
    response = client.chat.completions.create(
        model="anthropic/claude-3.5-sonnet",
        messages=[
            {"role": "user", "content": "Say 'Hello! Connection successful!' and nothing else."}
        ],
        max_tokens=50
    )
    
    print("Connection successful!")
    print(f"Response: {response.choices[0].message.content}")
    
except Exception as e:
    print(f"Connection failed: {e}")

Testing connection to OpenRouter...
Connection successful!
Response: Hello! Connection successful!


# Dummy Function for the LLM to call

In [6]:
from models.base import SessionLocal
from models import UseCase

In [7]:
def get_all_use_cases():
    """
    Get all use cases from the database.
    Returns a list of use cases with their basic info.
    """
    db = SessionLocal()
    try:
        use_cases = db.query(UseCase).all()
        
        # Format as a list of dictionaries
        result = []
        for uc in use_cases:
            result.append({
                "id": uc.id,
                "title": uc.title,
                "status": uc.status,
                "company": uc.company.name,
                "industry": uc.industry.name
            })
        
        return result
    finally:
        db.close()

In [ ]:
use_cases = get_all_use_cases()

2026-02-13 13:16:45,758 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-13 13:16:45,760 INFO sqlalchemy.engine.Engine SELECT use_cases.id AS use_cases_id, use_cases.title AS use_cases_title, use_cases.description AS use_cases_description, use_cases.expected_benefit AS use_cases_expected_benefit, use_cases.status AS use_cases_status, use_cases.company_id AS use_cases_company_id, use_cases.industry_id AS use_cases_industry_id 
FROM use_cases
2026-02-13 13:16:45,761 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ()
2026-02-13 13:16:45,763 INFO sqlalchemy.engine.Engine SELECT companies.id AS companies_id, companies.name AS companies_name, companies.industry_id AS companies_industry_id 
FROM companies 
WHERE companies.id = ?
2026-02-13 13:16:45,764 INFO sqlalchemy.engine.Engine [generated in 0.00084s] (1,)
2026-02-13 13:16:45,766 INFO sqlalchemy.engine.Engine SELECT industries.id AS industries_id, industries.name AS industries_name 
FROM industries 
WHERE industries.id = ?
202

In [9]:
use_cases

[{'id': 1,
  'title': 'Patient protal',
  'status': 'new',
  'company': 'MediCare Solutions Inc.',
  'industry': 'Healthcare'},
 {'id': 2,
  'title': 'AI Diagnostics',
  'status': 'in_progress',
  'company': 'HealthFirst Medical Group',
  'industry': 'Healthcare'},
 {'id': 3,
  'title': 'EHR Migration',
  'status': 'completed',
  'company': 'CarePlus Health Services',
  'industry': 'Healthcare'},
 {'id': 4,
  'title': 'Cloud Migration',
  'status': 'new',
  'company': 'DataStream Systems',
  'industry': 'IT'},
 {'id': 5,
  'title': 'Mobile App Dev',
  'status': 'in_progress',
  'company': 'TechVision Innovations',
  'industry': 'IT'},
 {'id': 6,
  'title': 'Cybersecurity Program',
  'status': 'new',
  'company': 'ByteForge Solutions',
  'industry': 'IT'},
 {'id': 7,
  'title': 'IoT Sensors',
  'status': 'in_progress',
  'company': 'Precision Parts Industries',
  'industry': 'Manufacturing'},
 {'id': 8,
  'title': 'Supply Chain Optimization',
  'status': 'new',
  'company': 'Global Fabr

# Try to make the LLM call the function 

In [12]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_all_use_cases",
            "description": "Get all use cases from the database. Use this when the user asks to see all use cases, list use cases, or show what use cases exist.",
            "parameters": {
                "type": "object",
                "properties": {},  # This function takes no parameters
                "required": []
            }
        }
    }
]


In [15]:
def run_agent(user_message):
    """
    Send a message to the agent and handle function calling.
    """
    print(f"\n{'='*60}")
    print(f"USER: {user_message}")
    print(f"{'='*60}")
    
    # send message to LLM and get response
    response = client.chat.completions.create(
        model="anthropic/claude-3.5-sonnet",
        messages=[
            {"role": "user", "content": user_message}
        ],
        tools=tools,  # pass tool created
        max_tokens=1000
    )
    
    # Step 2: Check if LLM wants to call a function
    message = response.choices[0].message
    
    if message.tool_calls:  # agant wants to call a function
        tool_call = message.tool_calls[0]
        function_name = tool_call.function.name
        
        print(f"\n AGENT: Calling function '{function_name}'...")
        
        # Step 3: Actually execute the function
        if function_name == "get_all_use_cases":
            function_result = get_all_use_cases()
        else:
            function_result = {"error": "Unknown function"}
        
        print(f"Function returned {len(function_result)} results")
        
        # Step 4: Send function result back to LLM
        second_response = client.chat.completions.create(
            model="anthropic/claude-3.5-sonnet",
            messages=[
                {"role": "user", "content": user_message},
                message,  # The assistant's tool call
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(function_result)
                }
            ],
            max_tokens=1000
        )
        
        final_answer = second_response.choices[0].message.content
        print(f"\n AGENT RESPONSE:\n{final_answer}")
        
    else:
        # LLM responded directly without calling a function
        print(f"\n AGENT RESPONSE:\n{message.content}")
    
    print(f"\n{'='*60}\n")

In [ ]:
run_agent("Show me all use cases")  # yay!


USER: Show me all use cases

 AGENT: Calling function 'get_all_use_cases'...
2026-02-13 13:24:14,615 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-13 13:24:14,615 INFO sqlalchemy.engine.Engine SELECT use_cases.id AS use_cases_id, use_cases.title AS use_cases_title, use_cases.description AS use_cases_description, use_cases.expected_benefit AS use_cases_expected_benefit, use_cases.status AS use_cases_status, use_cases.company_id AS use_cases_company_id, use_cases.industry_id AS use_cases_industry_id 
FROM use_cases
2026-02-13 13:24:14,616 INFO sqlalchemy.engine.Engine [cached since 448.9s ago] ()
2026-02-13 13:24:14,618 INFO sqlalchemy.engine.Engine SELECT companies.id AS companies_id, companies.name AS companies_name, companies.industry_id AS companies_industry_id 
FROM companies 
WHERE companies.id = ?
2026-02-13 13:24:14,618 INFO sqlalchemy.engine.Engine [cached since 448.9s ago] (1,)
2026-02-13 13:24:14,619 INFO sqlalchemy.engine.Engine SELECT industries.id AS industries_id

# Check if it doesnt call the function if not needed

In [ ]:
run_agent("What is a use case?")

# Agent still calls the function, yet decides to not include it in the answer...


USER: What is a use case?

 AGENT: Calling function 'get_all_use_cases'...
2026-02-13 13:26:12,575 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-13 13:26:12,577 INFO sqlalchemy.engine.Engine SELECT use_cases.id AS use_cases_id, use_cases.title AS use_cases_title, use_cases.description AS use_cases_description, use_cases.expected_benefit AS use_cases_expected_benefit, use_cases.status AS use_cases_status, use_cases.company_id AS use_cases_company_id, use_cases.industry_id AS use_cases_industry_id 
FROM use_cases
2026-02-13 13:26:12,578 INFO sqlalchemy.engine.Engine [cached since 566.8s ago] ()
2026-02-13 13:26:12,580 INFO sqlalchemy.engine.Engine SELECT companies.id AS companies_id, companies.name AS companies_name, companies.industry_id AS companies_industry_id 
FROM companies 
WHERE companies.id = ?
2026-02-13 13:26:12,580 INFO sqlalchemy.engine.Engine [cached since 566.8s ago] (1,)
2026-02-13 13:26:12,581 INFO sqlalchemy.engine.Engine SELECT industries.id AS industries_id, 

In [18]:
# Test 3: Count question
run_agent("How many use cases do we have?")


USER: How many use cases do we have?

 AGENT: Calling function 'get_all_use_cases'...
2026-02-13 13:28:33,659 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-13 13:28:33,661 INFO sqlalchemy.engine.Engine SELECT use_cases.id AS use_cases_id, use_cases.title AS use_cases_title, use_cases.description AS use_cases_description, use_cases.expected_benefit AS use_cases_expected_benefit, use_cases.status AS use_cases_status, use_cases.company_id AS use_cases_company_id, use_cases.industry_id AS use_cases_industry_id 
FROM use_cases
2026-02-13 13:28:33,662 INFO sqlalchemy.engine.Engine [cached since 707.9s ago] ()
2026-02-13 13:28:33,664 INFO sqlalchemy.engine.Engine SELECT companies.id AS companies_id, companies.name AS companies_name, companies.industry_id AS companies_industry_id 
FROM companies 
WHERE companies.id = ?
2026-02-13 13:28:33,664 INFO sqlalchemy.engine.Engine [cached since 707.9s ago] (1,)
2026-02-13 13:28:33,665 INFO sqlalchemy.engine.Engine SELECT industries.id AS indu

In [19]:
# Test 4: Filter by industry
run_agent("What use cases are in the healthcare industry?")


USER: What use cases are in the healthcare industry?

 AGENT: Calling function 'get_all_use_cases'...
2026-02-13 13:29:22,678 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-13 13:29:22,679 INFO sqlalchemy.engine.Engine SELECT use_cases.id AS use_cases_id, use_cases.title AS use_cases_title, use_cases.description AS use_cases_description, use_cases.expected_benefit AS use_cases_expected_benefit, use_cases.status AS use_cases_status, use_cases.company_id AS use_cases_company_id, use_cases.industry_id AS use_cases_industry_id 
FROM use_cases
2026-02-13 13:29:22,680 INFO sqlalchemy.engine.Engine [cached since 756.9s ago] ()
2026-02-13 13:29:22,682 INFO sqlalchemy.engine.Engine SELECT companies.id AS companies_id, companies.name AS companies_name, companies.industry_id AS companies_industry_id 
FROM companies 
WHERE companies.id = ?
2026-02-13 13:29:22,683 INFO sqlalchemy.engine.Engine [cached since 756.9s ago] (1,)
2026-02-13 13:29:22,684 INFO sqlalchemy.engine.Engine SELECT indus